<a href="https://colab.research.google.com/github/IlyaPvd/automl_course2025/blob/main/automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

import numpy as np
import pandas as pd
import warnings

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

warnings.filterwarnings('ignore')

# Configure pandas display settings for better data visualization
pd.options.display.max_columns = None

def preprocess_numerical_features(df):
    """
    Handle numerical features by converting strings to numbers and imputing missing values.
    Args:
        df: Input DataFrame
    Returns:
        DataFrame with processed numerical features
    """
    # Convert string columns to numeric, handling errors gracefully
    numeric_columns = ['person_emp_length', 'loan_int_rate']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Impute missing values using median strategy
    imputer = SimpleImputer(strategy='median')
    imputed_values = imputer.fit_transform(df[numeric_columns])
    df[numeric_columns] = pd.DataFrame(imputed_values, columns=numeric_columns, index=df.index)

    return df

def encode_categorical_features(df):
    """
    Convert categorical variables to appropriate data types and encode them.
    Args:
        df: Input DataFrame
    Returns:
        DataFrame with encoded categorical features
    """
    categorical_columns = [
        'person_home_ownership',
        'loan_intent',
        'loan_grade',
        'cb_person_default_on_file'
    ]

    for col in categorical_columns:
        df[col] = df[col].astype('category')

    return df

def engineer_credit_features(df):
    """
    Create advanced features for credit risk assessment.
    Includes debt-to-income ratios, risk indicators, and financial metrics.
    Args:
        df: Input DataFrame
    Returns:
        DataFrame with additional engineered features
    """
    # Calculate financial ratios and interactions
    df['debt_to_income'] = ((df['loan_amnt'] / df['person_income']) - df['loan_percent_income'])
    df['age_income_factor'] = (df['person_age'] * df['person_income'])
    df['loan_employment_ratio'] = (df['loan_amnt'] / df['person_emp_length'])

    # Monthly financial calculations
    monthly_income = df['person_income'] / 12
    df['monthly_loan_payment'] = (df['loan_amnt'] * (1 + df['loan_int_rate']) / 12)
    df['monthly_debt_ratio'] = (df['monthly_loan_payment'] / monthly_income)

    # Risk assessment feature
    df['credit_risk_indicator'] = pd.Series(
        np.where(
            (df['cb_person_default_on_file'] == 'Y') &
            (df['loan_grade'].isin(['C', 'D', 'E'])),
            1, 0
        ),
        index=df.index
    )

    # Convert features to category type
    categorical_features = [
        'debt_to_income', 'age_income_factor', 'loan_employment_ratio',
        'monthly_loan_payment', 'monthly_debt_ratio', 'credit_risk_indicator'
    ]
    for col in categorical_features:
        df[col] = df[col].astype(str).astype('category')

    # Standardize categorical encodings
    categorical_mappings = {
        'person_home_ownership': {'RENT': 0, 'MORTGAGE': 1, 'OWN': 2, 'OTHER': 3},
        'loan_intent': {'EDUCATION': 0, 'MEDICAL': 1, 'PERSONAL': 2, 'VENTURE': 3,
                       'DEBTCONSOLIDATION': 4, 'HOMEIMPROVEMENT': 5},
        'loan_grade': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6},
        'cb_person_default_on_file': {'N': 0, 'Y': 1}
    }

    for col, mapping in categorical_mappings.items():
        df[col] = df[col].replace(mapping)

    # Normalize loan-to-income ratio
    df["normalized_loan_income_ratio"] = (((df["loan_amnt"] / df["person_income"])) * 1000000).astype(int)

    return df

# Load datasets
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
sample = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv', index_col=0)
supplementary_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/credit_risk_dataset.csv')

# Initial data preparation
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)
train = pd.concat(objs=[train, supplementary_data])

# Apply preprocessing steps
train = preprocess_numerical_features(train)
train = encode_categorical_features(train)
train = engineer_credit_features(train)

test = encode_categorical_features(test)
test = engineer_credit_features(test)

# Initialize binary classification task
task = Task('binary')

# Train ensemble model with cross-validation
predictions = []
for i in range(5):
    X_train, X_val = train_test_split(
        train,
        test_size=0.15+0.05*i,
        random_state=5*i+42*i*i*i,
        shuffle=True,
        stratify=train['loan_status']
    )

    model = TabularAutoML(
        task=task,
        timeout=600 * 3600,
        cpu_limit=24,
        gpu_ids='0',
        reader_params={'n_jobs': 6, 'cv': 5, 'random_state': 5*i+42*i*i*i}
    )

    model.fit_predict(
        X_train,
        valid_data=X_val,
        roles={'target': 'loan_status'},
        verbose=1
    )

    pred = model.predict(test).data[:, 0]
    predictions.append(list(pred))

# Generate ensemble predictions
ensemble_predictions = [sum(col) / len(col) for col in zip(*predictions)]
sample['loan_status'] = ensemble_predictions
sample.to_csv('submission.csv')

# Train final neural network model
X_train, X_val = train_test_split(
    train,
    test_size=0.3,
    random_state=3407,
    shuffle=True,
    stratify=train['loan_status']
)

nn_model = TabularAutoML(
    task=task,
    timeout=600 * 3600,
    cpu_limit=24,
    gpu_ids='0',
    general_params={"use_algos": [['denselight']]},
    nn_params={
        "n_epochs": 10,
        "bs": 128,
        "num_workers": 0,
        "path_to_save": None,
        "freeze_defaults": True,
        "cont_embedder": 'plr',
        'cat_embedder': 'weighted',
        'act_fun': 'SiLU',
        "hidden_size": [512, 256],
        'stop_by_metric': True,
        'embedding_size': 256,
        'verbose_bar': True,
        "snap_params": {'k': 2, 'early_stopping': True, 'patience': 1, 'swa': True},
        'opt_params': {'lr': 0.0003, 'weight_decay': 0}
    },
    nn_pipeline_params={"use_qnt": True, "use_te": False},
    reader_params={'n_jobs': 12, 'cv': 5, 'random_state': 42, 'advanced_roles': True}
)

nn_model.fit_predict(
    X_train,
    valid_data=X_val,
    roles={'target': 'loan_status'},
    verbose=4
)

# Generate neural network predictions
nn_predictions = nn_model.predict(test).data[:, 0]
sample['loan_status'] = nn_predictions
sample.to_csv('submission.csv')


[08:11:39] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[08:11:39] Task: binary



INFO:lightautoml.automl.presets.base:Task: binary



[08:11:39] Start automl preset with listed constraints:


...

INFO:lightautoml.automl.base:Layer 1 training completed.



[09:07:54] Automl preset training completed in 37.48 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 37.48 seconds



[09:07:54] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (1 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (1 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0) 

test: 100%|██████████| 306/306 [00:01<00:00, 286.94it/s]


In [1]:
!pip install lightautoml

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.6/399.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.0/216.0 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.9 MB/s eta 0:00:00
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7594 sha256=8e9a66b4e9baddb729e9adeff3462622794ce743e319e0e9777ff2c86b967583
  St

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
